In [1]:
!pip install tensorflow opencv-python matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp /content/drive/MyDrive/Dataset/food_data.zip .

In [4]:
import zipfile

with zipfile.ZipFile("food_data.zip", 'r') as zip_ref:
    zip_ref.extractall("food_data")

import shutil
shutil.rmtree("food_data/__MACOSX", ignore_errors=True)
shutil.rmtree("food_data/food_data", ignore_errors=True)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 100

# This is where you add the augmentation parameters.
# `datagen` for training data includes augmentations and rescaling.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# This is a separate datagen for validation, with only rescaling.
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Use the train_datagen for your training data generator
train_data = train_datagen.flow_from_directory(
    'food_data',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Use the val_datagen for your validation data generator
val_data = val_datagen.flow_from_directory(
    'food_data',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# img_size = 100

# datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# train_data = datagen.flow_from_directory('food_data',target_size=(img_size, img_size),batch_size=32,class_mode='categorical',subset='training')

# val_data = datagen.flow_from_directory('food_data',target_size=(img_size, img_size),batch_size=32,class_mode='categorical',subset='validation')

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(train_data.num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,333,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,354,378 (16.61 MB)

 Trainable params: 4,354,378 (16.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Create the EarlyStopping callback
# It will monitor 'val_loss' and stop if it doesn't improve for 5 consecutive epochs
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Now, add this callback to your model.fit() function
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=25, # Set the number of epochs to 25
    callbacks=[early_stopping] # Add the callback here
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 45s 163ms/step - accuracy: 0.1334 - loss: 2.3401 - val_accuracy: 0.2220 - val_loss: 2.0988
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 157ms/step - accuracy: 0.2117 - loss: 2.1362 - val_accuracy: 0.2495 - val_loss: 2.0410
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.2378 - loss: 2.0910 - val_accuracy: 0.2910 - val_loss: 1.9851
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.2665 - loss: 2.0378 - val_accuracy: 0.3265 - val_loss: 1.9035
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.2813 - loss: 1.9984 - val_accuracy: 0.3350 - val_loss: 1.9250
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.3045 - loss: 1.9532 - val_accuracy: 0.3450 - val_loss: 1.8559
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 155ms/step - accuracy: 0.3277 - loss: 1.9033 - val_accuracy: 0.3750 - val_loss: 1.8172
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.3383 - loss: 1

In [ ]:
# history = model.fit(train_data, validation_data=val_data, epochs=25)

In [8]:
from google.colab import files
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# Upload and rename file
uploaded = files.upload()

for fname in uploaded.keys():
    os.rename(fname, "test.jpg")

# Load and preprocess
img = image.load_img("test.jpg", target_size=(img_size, img_size))
img_arr = image.img_to_array(img) / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

# Predict
pred = model.predict(img_arr)[0]
class_index = np.argmax(pred)
confidence = pred[class_index] * 100
label_map = list(train_data.class_indices.keys())

# Print result
print("\n----------------------------------------------------------------------")
print(f"\nPredicted food: {label_map[class_index]}")
print(f"Confidence: {confidence:.2f}%")

Saving images.jpeg to images.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step

----------------------------------------------------------------------

Predicted food: tacos
Confidence: 48.93%


In [9]:
# Save your trained model
model.save("food_classifier_model.keras")

In [13]:
# Add this cell at the END of your Jupyter notebook after training is complete

import os
import subprocess
import sys
import tensorflow as tf
import json

# Install tensorflowjs if not already installed
try:
    import tensorflowjs as tfjs
    print("TensorFlow.js already installed")
except ImportError:
    print("Installing TensorFlow.js...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflowjs"])
    import tensorflowjs as tfjs

# Your food categories (matching your actual dataset)
food_categories = [
    'chocolate_cake',
    'donuts',
    'french_fries',
    'grilled_cheese_sandwich',
    'ice_cream',
    'pancakes',
    'pizza',
    'samosa',
    'sushi',
    'tacos'
]

print("Starting model conversion for web deployment...")
print("=" * 60)

# Display food categories for verification
print(f"\nFood Categories ({len(food_categories)} classes):")
for i, category in enumerate(food_categories):
    print(f"   {i}: {category}")

# Convert model
try:
    print(f"\nConverting model for web deployment...")

    # Create output directory
    output_dir = "model"
    os.makedirs(output_dir, exist_ok=True)

    # Load your trained model
    print(f"Loading model: /content/food_classifier_model.keras")
    model = tf.keras.models.load_model('/content/food_classifier_model.keras')

    # Display model info
    print(f"Model Summary:")
    print(f"   - Input shape: {model.input_shape}")
    print(f"   - Output shape: {model.output_shape}")
    print(f"   - Total parameters: {model.count_params():,}")

    print(f"\nConverting to TensorFlow.js format...")

    # Convert the model
    tfjs.converters.save_keras_model(model, output_dir)

    print(f"Model converted successfully!")
    print(f"Generated files in '{output_dir}' folder:")

    # Show generated files and sizes
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        size = os.path.getsize(file_path)
        print(f"   - {file} ({size:,} bytes)")

    # Save food categories as JSON for the web app
    import json
    categories_path = os.path.join(output_dir, "food_categories.json")
    with open(categories_path, 'w') as f:
        json.dump(food_categories, f, indent=2)
    print(f"Food categories saved to: {categories_path}")

    print(f"\nModel conversion completed successfully!")
    print(f"Your model is ready for web deployment!")

except Exception as e:
    print(f"Error during conversion: {str(e)}")
    print("Make sure your model file exists at /content/food_classifier_model.keras")

TensorFlow.js already installed
Starting model conversion for web deployment...

Food Categories (10 classes):
   0: chocolate_cake
   1: donuts
   2: french_fries
   3: grilled_cheese_sandwich
   4: ice_cream
   5: pancakes
   6: pizza
   7: samosa
   8: sushi
   9: tacos

Converting model for web deployment...
Loading model: /content/food_classifier_model.keras


Model Summary:
   - Input shape: (None, 100, 100, 3)
   - Output shape: (None, 10)
   - Total parameters: 4,354,378

Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
Model converted successfully!
Generated files in 'model' folder:
   - model.json (5,675 bytes)
   - group1-shard1of5.bin (4,194,304 bytes)
   - group1-shard4of5.bin (4,194,304 bytes)
   - group1-shard5of5.bin (640,296 bytes)
   - group1-shard2of5.bin (4,194,304 bytes)
   - group1-shard3of5.bin (4,194,304 bytes)
Food categories saved to: model/food_categories.json

Model conversion completed successfully!
Your model is ready for web deployment!
